In [22]:
import tensorflow as tf
import numpy as np
import gym
from collections import deque
import random
import warnings
import cv2
warnings.filterwarnings('ignore')
from skimage import transform
from skimage.color import rgb2gray
# Test the game

In [3]:
env = gym.make('SpaceInvaders-v0')
env.reset()
for _ in range(3000):
    env.step(env.action_space.sample())
    env.render('human')
env.close()


# Getting to Know the game

In [4]:
print(env.observation_space)
print(env.action_space.n)

Box(210, 160, 3)
6


# Preprocessing
To remove unnecessary parts of the frame

In [23]:
def preprocess_frame(frame):
    # Greyscale frame 
    gray = rgb2gray(frame)
    
    # Crop the screen (remove the part below the player)
    # [Up: Down, Left: right]
    cropped_frame = gray[8:-12,4:-12]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    # Thanks to Mikołaj Walkowiak
    preprocessed_frame = transform.resize(normalized_frame, [110,84])
    
    return preprocessed_frame # 110x84x1 frame

In [24]:
stack_size = 4

def stack_frames(stacked_frames, state, is_new_episode):
    frame = preprocess_frame(state)
    
    if is_new_episode:
        stacked_frames = deque([np.zeros((110, 84), dtype=np.int) for _ in range(stack_size)], maxlen=stack_size)
        for _ in range(stack_size):
            stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
    
    else:
        stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)

    return  stacked_state, stacked_frames


# Hyper Params